In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1 Loading the data
NB: the json need to be a list of json elements as dictionary. We need to add the **brackets []** and the **coma** to separate each element

In [2]:
with open("../yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json") as json_file:
    data_business = json.load(json_file)


In [3]:
data_business[35]

{u'attributes': {u'Accepts Credit Cards': True,
  u'Alcohol': u'full_bar',
  u'Ambience': {u'casual': True,
   u'classy': False,
   u'divey': False,
   u'hipster': False,
   u'intimate': False,
   u'romantic': False,
   u'touristy': False,
   u'trendy': False,
   u'upscale': False},
  u'Attire': u'casual',
  u'BYOB': False,
  u'BYOB/Corkage': u'yes_free',
  u'Caters': False,
  u'Corkage': False,
  u'Delivery': False,
  u'Good For': {u'breakfast': False,
   u'brunch': False,
   u'dessert': False,
   u'dinner': False,
   u'latenight': True,
   u'lunch': False},
  u'Good For Groups': True,
  u'Good For Kids': False,
  u'Good for Kids': True,
  u'Has TV': True,
  u'Noise Level': u'very_loud',
  u'Outdoor Seating': False,
  u'Parking': {u'garage': False,
   u'lot': True,
   u'street': False,
   u'valet': False,
   u'validated': False},
  u'Price Range': 2,
  u'Take-out': True,
  u'Takes Reservations': True,
  u'Waiter Service': True,
  u'Wheelchair Accessible': True},
 u'business_id': u'sbW

Somes features need to be droped:
* 'full_address', 'city': the state and latitude/longitude already stand for the geographical position. Maybe the street could eb considered, but then will need to be convert...
* 'name': need to find how to use it (add features count to find a word realted to categories in the name?)
* 'type': stands for the data type (here it's business)

Somes features are droped but may be useful, we need to think about how to use them:
* 'attributes': issue is the unstructured nature
* 'hours': the checkin seems to be enough
* 'neighborhoods': many missing values in this column

In [4]:
business_list = []
    
for b in data_business:
    d = b.copy()
    
    # Dropping unconsidered features
    d.pop('hours')
    #d.pop('attributes')
    d.pop('neighborhoods')
    d.pop('type')
    d.pop('full_address')
    d.pop('name')
    #d.pop('city')
    
    
    business_list.append(d)
    
business=pd.DataFrame(business_list)
# Changing the index
business = business.set_index('business_id')

business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61184 entries, vcNAWiLM4dR7D2nwwJ7nCA to 2EKGrbf2_81MrtjKZeOTng
Data columns (total 9 columns):
attributes      61184 non-null object
categories      61184 non-null object
city            61184 non-null object
latitude        61184 non-null float64
longitude       61184 non-null float64
open            61184 non-null bool
review_count    61184 non-null int64
stars           61184 non-null float64
state           61184 non-null object
dtypes: bool(1), float64(3), int64(1), object(4)
memory usage: 4.3+ MB


# City filtering
First test with Las Vegas

In [5]:
# Load the business id of Las Vegas restaurants
lasvegas_id = np.load('temp/lasvegas_id.npy')
len(lasvegas_id)

3822

In [6]:
# Filtering the businesses
business_lasvegas = business[business.index.isin(lasvegas_id)]
print len(business_lasvegas)
business_lasvegas.head(2)

3822


,attributes,categories,city,latitude,longitude,open,review_count,stars,state
business_id,,,,,,,,,
l6QcUE8XXLrVH6Ydm4GSNw,"{u'Drive-Thru': False, u'Alcohol': u'none', u'...","[Burgers, American (Traditional), Fast Food, R...",Las Vegas,36.197365,-115.191711,True,13,3,NV
AgtFQVxo-PGQVsOdY6Nvdg,"{u'Take-out': True, u'Alcohol': u'none', u'Has...","[Pizza, Restaurants]",Las Vegas,36.173110,-115.125401,True,3,4,NV


# Checkin data

In [7]:
with open("../yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_checkin.json") as json_file:
    data_checkin = json.load(json_file)


In [8]:
data_checkin[0]

{u'business_id': u'cE27W9VPgO88Qxe4ol6y_g',
 u'checkin_info': {u'10-0': 1,
  u'13-0': 1,
  u'13-3': 1,
  u'14-6': 1,
  u'17-3': 1,
  u'17-6': 1,
  u'18-4': 1,
  u'7-5': 1,
  u'9-5': 1},
 u'type': u'checkin'}

In [9]:
checkin_list = []

for c in data_checkin:
    d = {}
    d['business_id'] = c['business_id']
    for k,v in c['checkin_info'].iteritems():
        d[k] = v
    checkin_list.append(d)
    
checkin=pd.DataFrame(checkin_list)
checkin = checkin.set_index('business_id')

# Change Nan to 0 in the checking
checkin = checkin.fillna(0)

checkin.head(10)

,0-0,0-1,0-2,0-3,0-4,0-5,0-6,1-0,1-1,1-2,...,8-4,8-5,8-6,9-0,9-1,9-2,9-3,9-4,9-5,9-6
business_id,,,,,,,,,,,,,,,,,,,,,
cE27W9VPgO88Qxe4ol6y_g,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
mVHrayjG3uZ_RLHkLj-AMg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
KayYbHCt-RkbGcPdGOThNg,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,3,0,0
wJr6kSA5dchdgOdwH6dZ2w,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
yXuao0pFz1AxB21vJjDf5w,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
fNGIbpazjTRdXgwRY_NIXA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
b9WZJp5L1RZr4F1nxclOoQ,0,0,0,0,0,0,0,0,0,0,...,2,3,4,2,3,1,2,4,4,1
zaXDakTd3RXyOa7sMrUE1g,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,1,0,0
zgy27FSnvwdINfk5cXBIyQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
# ## Joining the dataframes
business_lasvegas_df = business_lasvegas.merge(checkin, how='left', right_index=True, left_index=True)
business_lasvegas_df.head(2)

,attributes,categories,city,latitude,longitude,open,review_count,stars,state,0-0,...,8-4,8-5,8-6,9-0,9-1,9-2,9-3,9-4,9-5,9-6
business_id,,,,,,,,,,,,,,,,,,,,,
l6QcUE8XXLrVH6Ydm4GSNw,"{u'Drive-Thru': False, u'Alcohol': u'none', u'...","[Burgers, American (Traditional), Fast Food, R...",Las Vegas,36.197365,-115.191711,True,13,3,NV,0,...,0,0,0,0,0,0,0,0,0,0
AgtFQVxo-PGQVsOdY6Nvdg,"{u'Take-out': True, u'Alcohol': u'none', u'Has...","[Pizza, Restaurants]",Las Vegas,36.173110,-115.125401,True,3,4,NV,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Droping the business with Nan values of checkin
business_lasvegas_df = business_lasvegas_df.dropna(how='any')
len(business_lasvegas_df)

3822

In [12]:
business_lasvegas_df.categories

business_id
l6QcUE8XXLrVH6Ydm4GSNw    [Burgers, American (Traditional), Fast Food, R...
AgtFQVxo-PGQVsOdY6Nvdg                                 [Pizza, Restaurants]
AFDFOs2GaIyUArGyEUDQzQ                               [Chinese, Restaurants]
FV16IeXJp2W6pnghTz2FAw                   [Vietnamese, Chinese, Restaurants]
cvJDHk1ho0DxaF26etcr8Q                        [Latin American, Restaurants]
10UcN8MF1kQiyTJAfHKvfQ            [Delis, Italian, Sandwiches, Restaurants]
R5e2XWtpa_CA77MIXEyCTg                 [Fast Food, Sandwiches, Restaurants]
CoJivG0wxNIBXyOF7MNqSQ                            [Vietnamese, Restaurants]
pCobXgTy6DIfBaT4pjlq0w                            [Sandwiches, Restaurants]
JgpKLikZPbbXYPtILy3-wg               [Latin American, Mexican, Restaurants]
9-O8qKGQQCYXdBlj4iv7bw                               [Restaurants, Italian]
PsOvURXqpy8iZX2J8y429A                               [Seafood, Restaurants]
Yq8LiVymGA7vBpGCQuDfRw                            [Vietnamese, Restaurants]


## 2 Restriction on restaurants: 

In [11]:
# Restricting to the restaurants
restaurants = business_df[business_df['categories'].apply(lambda x: 'Restaurants' in x)]

In [12]:
restaurants.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18640 entries, mVHrayjG3uZ_RLHkLj-AMg to qrgod2lrYnxHwNWC_YYTyQ
Columns: 176 entries, attributes to 9-6
dtypes: bool(1), float64(171), int64(1), object(3)
memory usage: 25.0+ MB


# Categories

In [13]:
from collections import Counter

restaurant_categories = business_lasvegas_df.categories.values
categories_list = [c for cs in restaurant_categories for c in cs]
c = Counter(categories_list)
restaurant_categories_set = set(categories_list)
len(restaurant_categories_set)

# Keeping only the top categories (appear more than 10 times)
top_categories = {k: v for k, v in c.iteritems() if v>10} 
len(top_categories)

# Get ridd of 'Restaurants' category
top_categories.pop('Restaurants')

top_categories

{u'American (New)': 280,
 u'American (Traditional)': 403,
 u'Arts & Entertainment': 45,
 u'Asian Fusion': 91,
 u'Bagels': 12,
 u'Bakeries': 36,
 u'Barbeque': 93,
 u'Bars': 322,
 u'Beer, Wine & Spirits': 19,
 u'Breakfast & Brunch': 156,
 u'Buffets': 88,
 u'Burgers': 296,
 u'Cafes': 97,
 u'Cajun/Creole': 16,
 u'Casinos': 25,
 u'Chicken Wings': 85,
 u'Chinese': 286,
 u'Cocktail Bars': 14,
 u'Coffee & Tea': 43,
 u'Delis': 85,
 u'Desserts': 32,
 u'Dim Sum': 15,
 u'Diners': 49,
 u'Event Planning & Services': 20,
 u'Fast Food': 553,
 u'Filipino': 37,
 u'Food': 257,
 u'Food Delivery Services': 11,
 u'Food Stands': 33,
 u'Food Trucks': 20,
 u'French': 31,
 u'Gluten-Free': 20,
 u'Greek': 32,
 u'Grocery': 11,
 u'Hawaiian': 57,
 u'Hot Dogs': 57,
 u'Ice Cream & Frozen Yogurt': 16,
 u'Indian': 26,
 u'Italian': 232,
 u'Japanese': 201,
 u'Juice Bars & Smoothies': 29,
 u'Karaoke': 13,
 u'Korean': 65,
 u'Latin American': 29,
 u'Lounges': 41,
 u'Mediterranean': 68,
 u'Mexican': 476,
 u'Middle Eastern': 2

In [14]:
# Keeping only the best categories

def filter_categories(categories, top_categories):
    new_categories = []
    for c in categories:
        if c in top_categories:
            new_categories.append(c)
    return new_categories

business_lasvegas_df['categories_top'] = business_lasvegas_df.apply(lambda x: filter_categories(x['categories'], top_categories.keys()), axis=1)


In [15]:
# Removing the restaurants without categories in top categories

business_lasvegas_df['num_cat'] = business_lasvegas_df.apply(lambda x: len(x['categories_top']), axis=1)
restaurants_lasvegas = business_lasvegas_df[business_lasvegas_df.num_cat > 0]

business_lasvegas_df

,attributes,categories,city,latitude,longitude,open,review_count,stars,state,0-0,...,8-6,9-0,9-1,9-2,9-3,9-4,9-5,9-6,categories_top,num_cat
business_id,,,,,,,,,,,,,,,,,,,,,
l6QcUE8XXLrVH6Ydm4GSNw,"{u'Drive-Thru': False, u'Alcohol': u'none', u'...","[Burgers, American (Traditional), Fast Food, R...",Las Vegas,36.197365,-115.191711,True,13,3.0,NV,0,...,0,0,0,0,0,0,0,0,"[Burgers, American (Traditional), Fast Food]",3
AgtFQVxo-PGQVsOdY6Nvdg,"{u'Take-out': True, u'Alcohol': u'none', u'Has...","[Pizza, Restaurants]",Las Vegas,36.173110,-115.125401,True,3,4.0,NV,0,...,0,0,0,0,0,0,0,0,[Pizza],1
AFDFOs2GaIyUArGyEUDQzQ,"{u'Alcohol': u'beer_and_wine', u'Noise Level':...","[Chinese, Restaurants]",Las Vegas,36.125908,-115.197904,True,155,3.0,NV,13,...,0,0,0,0,0,0,0,1,[Chinese],1
FV16IeXJp2W6pnghTz2FAw,"{u'Take-out': True, u'Accepts Credit Cards': T...","[Vietnamese, Chinese, Restaurants]",Las Vegas,36.126177,-115.193971,True,1514,3.5,NV,51,...,26,18,11,19,10,9,27,41,"[Vietnamese, Chinese]",2
cvJDHk1ho0DxaF26etcr8Q,"{u'Take-out': True, u'Accepts Credit Cards': T...","[Latin American, Restaurants]",Las Vegas,36.142935,-115.207747,True,48,3.5,NV,0,...,0,0,0,0,0,0,0,0,[Latin American],1
10UcN8MF1kQiyTJAfHKvfQ,{u'Accepts Credit Cards': True},"[Delis, Italian, Sandwiches, Restaurants]",Las Vegas,36.145672,-115.187809,True,3,5.0,NV,0,...,0,0,0,0,0,1,0,0,"[Delis, Italian, Sandwiches]",3
R5e2XWtpa_CA77MIXEyCTg,"{u'Delivery': False, u'Accepts Credit Cards': ...","[Fast Food, Sandwiches, Restaurants]",Las Vegas,36.130335,-115.206002,True,5,2.5,NV,0,...,0,0,0,0,0,0,0,0,"[Fast Food, Sandwiches]",2
CoJivG0wxNIBXyOF7MNqSQ,"{u'Take-out': True, u'Accepts Credit Cards': T...","[Vietnamese, Restaurants]",Las Vegas,36.126378,-115.196994,True,65,3.0,NV,0,...,0,0,0,0,1,0,2,1,[Vietnamese],1
pCobXgTy6DIfBaT4pjlq0w,"{u'Take-out': True, u'Accepts Credit Cards': T...","[Sandwiches, Restaurants]",Las Vegas,36.143005,-115.208009,True,32,3.0,NV,0,...,8,12,8,7,6,8,17,9,[Sandwiches],1


# Attributes

In [17]:
# Study on the attributes

attributes_keys = [k for attributes in restaurants_lasvegas.attributes.values for k in attributes.keys()]
len(attributes_keys)

counter_att = Counter(attributes_keys)

# Keep the top attributes
top_attributes = {k: v for k, v in counter_att.iteritems() if v > len(restaurants_lasvegas)/2} 
print len(top_attributes)
top_attributes

19


{u'Accepts Credit Cards': 3664,
 u'Alcohol': 3313,
 u'Ambience': 3250,
 u'Attire': 3591,
 u'Caters': 2804,
 u'Delivery': 3535,
 u'Good For': 3442,
 u'Good For Groups': 3605,
 u'Good for Kids': 3618,
 u'Has TV': 3395,
 u'Noise Level': 3199,
 u'Outdoor Seating': 3531,
 u'Parking': 3343,
 u'Price Range': 3632,
 u'Take-out': 3615,
 u'Takes Reservations': 3567,
 u'Waiter Service': 3356,
 u'Wheelchair Accessible': 2018,
 u'Wi-Fi': 2920}

In [18]:
# Values for different attributes
attributes_dict = []
def get_attribute(attributes, attribute, attributes_dict):
    if attribute in attributes:
        val = attributes[attribute]
        if not isinstance(val, dict):
            return val
        else:
            attributes_dict.append(attribute)
            return 'dict'
    return np.nan
                
for attribute in top_attributes:
    restaurants_lasvegas[attribute] = restaurants_lasvegas.apply(lambda x: get_attribute(x.attributes, attribute, attributes_dict) , axis=1)


/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
# Removing Attributes which are dictionary
attributes_dict_set = set(attributes_dict)

for at in attributes_dict_set:
    #restaurants_lasvegas.pop(at)
    top_attributes.pop(at)

KeyError: u'Ambience'

In [30]:
# Handling missing values in attributes: replacing with the most frequent value
fill_attribute_dict = {}
for attribute in top_attributes:
    fill_attribute_dict[attribute] = restaurants_lasvegas[attribute].value_counts().idxmax()
restaurants_lasvegas = restaurants_lasvegas.fillna(fill_attribute_dict)

In [36]:
# To find categorical features
def is_categorical(unique):
    if len(unique) > 2 and not isinstance(unique[0], float) and not isinstance(unique[0], bool) and not isinstance(unique[0], int):
        return True
    else:
        return False

# Listing the categorical features
categorical = []
for col in restaurants_lasvegas.columns.values:
    if col == 'attributes' or col=='categories' or col=='categories_top':
        continue
    if is_categorical(restaurants_lasvegas[col].unique()):
        categorical.append(col)
categorical

[u'Wi-Fi', u'Alcohol', u'Noise Level', u'Attire']

In [37]:
# Handling categorical feature: state
for c in categorical:
    restaurants_lasvegas[c] = restaurants_lasvegas[c].astype('category')
    dummies = pd.get_dummies(restaurants_lasvegas[c])
    restaurants_lasvegas = restaurants_lasvegas.join(dummies)
    restaurants_lasvegas = restaurants_lasvegas.drop([c], axis=1)

In [40]:
restaurants_lasvegas = restaurants_lasvegas.drop(['attributes'], axis=1)

In [52]:
restaurants_lasvegas['review_count'].max()

4578

# Supervised learning 

In [44]:
from sklearn import linear_model, datasets

# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
Y = iris.target
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [24]:
# Drop irrelevant features
data_exp = restaurants_clean.reset_index()
data_exp = data_exp.drop(['business_id', 'categories', 'attributes', 'target'], axis=1)

# Split train/test set
mask = np.random.rand(len(data_exp)) < 0.8
train_set = data_exp[mask]
test_set = data_exp[~mask]

print test_set.target_top.value_counts()
print train_set.target_top.value_counts()


0    2255
1     301
2     196
dtype: int64
0    8895
1    1250
2     810
dtype: int64


In [60]:
# Multiclass Logistic Regression
from sklearn.linear_model import LogisticRegression


y = train_set['target_top']
X = train_set.drop(['target_top'], axis=1)

lreg = LogisticRegression(penalty='l1', solver='lbfgs', multi_class='ovr')
clf = lreg.fit(X, y)

# Testing

y_test = test_set['target_top']
X_test = test_set.drop(['target_top'], axis=1)

y_test_pred = clf.predict(X_test)
y_score = clf.decision_function(X_test)


AttributeError: 'DummyClassifier' object has no attribute 'decision_function'

In [57]:
# PCA 
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression


# Dimensionality reduction
pca = PCA(n_components=0.999)
data_pca = pca.fit_transform(data_exp.drop(['target_top'], axis=1))

train_pca_set = data_pca[mask]
test_pca_set = data_pca[~mask]

y = train_set['target_top']
X = train_pca_set

lreg = LogisticRegression(penalty='l1', solver='lbfgs', multi_class='ovr')
clf = lreg.fit(X, y)

# Testing

y_test = test_set['target_top']
X_test = test_pca_set

y_test_pred = clf.predict(X_test)
y_score_pca = clf.decision_function(X_test)

In [46]:
# Plotting the confusion matrix

from sklearn.metrics import confusion_matrix

target_names = ['None'] + classes_name

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')



# Compute confusion matrix
cm = confusion_matrix(y_test, y_test_pred)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[2245    2    8]
 [ 296    5    0]
 [ 193    0    3]]
Normalized confusion matrix

/usr/local/lib/python2.7/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/usr/local/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):



[[  9.96e-01   8.87e-04   3.55e-03]
 [  9.83e-01   1.66e-02   0.00e+00]
 [  9.85e-01   0.00e+00   1.53e-02]]


In [84]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# Binarize the output
y_bin = label_binarize(y, classes=classes_s)
y_test_bin = label_binarize(y_test, classes=classes_s)
n_classes = y_bin.shape[1]

# Names of the classes
target_names = ['None'] + classes_name

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(target_names)):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot ROC curve
plt.figure()
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                   ''.format(target_names[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('multi-class ROC curve')
plt.legend(loc="lower right")
plt.show()

In [81]:
# Metrics and comparaison with dummy models (always predicting 0 for example)

from sklearn import metrics

print metrics.precision_score(y_test, np.zeros(y_test.shape[0]))
print metrics.precision_score(y_test, y_test_pred)


0.671423029551
0.726111608768


/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1082: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1082: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sampl

In [63]:
y_score

array([[ 1.8 , -2.28, -3.34],
       [ 2.4 , -2.73, -4.05],
       [ 1.33, -2.99, -1.83],
       ..., 
       [ 1.6 , -1.06, -4.09],
       [ 0.63, -1.89, -1.51],
       [ 1.34, -2.45, -2.26]])

In [69]:
len(restaurant_categories_set)

261

In [70]:
X.shape

(10955, 205)

## 3 Test against largest categories

In [89]:
# Study the top categories
import collections

categories_keys = [k for cat in business_df.categories.values for k in cat]
len(categories_keys)

counter2 = collections.Counter(categories_keys)
counter2.most_common(40)

[(u'Restaurants', 18640),
 (u'Food', 6637),
 (u'Shopping', 6504),
 (u'Nightlife', 3760),
 (u'Beauty & Spas', 3374),
 (u'Bars', 3205),
 (u'Fast Food', 2145),
 (u'Mexican', 2045),
 (u'Fashion', 2004),
 (u'Active Life', 1980),
 (u'American (Traditional)', 1968),
 (u'Automotive', 1958),
 (u'Pizza', 1828),
 (u'Health & Medical', 1828),
 (u'Sandwiches', 1796),
 (u'Event Planning & Services', 1776),
 (u'Arts & Entertainment', 1638),
 (u'Coffee & Tea', 1636),
 (u'Hotels & Travel', 1582),
 (u'Burgers', 1388),
 (u'American (New)', 1374),
 (u'Italian', 1327),
 (u'Chinese', 1265),
 (u'Grocery', 1107),
 (u'Hotels', 1103),
 (u'Breakfast & Brunch', 1036),
 (u'Nail Salons', 1023),
 (u'Local Services', 1008),
 (u'Hair Salons', 898),
 (u'Home & Garden', 828),
 (u'Fitness & Instruction', 792),
 (u'Specialty Food', 773),
 (u'Ice Cream & Frozen Yogurt', 764),
 (u'Bakeries', 743),
 (u'Home Services', 727),
 (u'Pets', 696),
 (u'Auto Repair', 693),
 (u"Women's Clothing", 690),
 (u'Sports Bars', 682),
 (u'Pubs

In [88]:
business_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45166 entries, cE27W9VPgO88Qxe4ol6y_g to 7gmzKhAr6-A0DCGT4HN-ig
Columns: 176 entries, attributes to 9-6
dtypes: bool(1), float64(171), int64(1), object(3)
memory usage: 60.7+ MB
